### Why we might need a Relational DB for Machine Learning?

There are many situations where you may need to store some stages of your data pipeline in a sql DB
For example: Your pipeline generates a aggregate table of few million rows on which business users want to run fast queries and you want to use such tables for machine learning. Now storing data in a sql table gives you fast querying as well as auditing so its a good choice.

Another situation arises when your machine learning algorithm makes certain predictions. You need to show these predictions to your customers as such this needs to be accessed by a web service. Also SQL interface is the best interface for adhoc querying. Hence storing in a sql db makes sense. 



#### When you can use sql db with python in your data pipeline?

**Scenario 1:**

Any Regression or classification problem

- Data from HIVE/Hadoop job is pushed into Sql DB.
- Your ML Algorithm needs this data so you export this as csv and use `pandas.read_csv`
- You generate predictions as a csv and hand over the csv to dev team by storing in AWS S3 or other network file storage systems.
- Your dev team puts the predictions in some Db for production usage.

**Alternate Improved Scenario for above**

- Data from HIVE/Hadoop job is pushed into Sql DB.
- Your ML Algorithm needs this data so you connect to the DB with python and read latest data.
- You generate predictions and put the predictions into the DB again using python.
- Finally you automate these steps and move on.

Notice how in second case: No dev team is involved and no need of network file storage. Data used is always fresh. Automation is easier since less components are involved


**Scenario 2:**

A specific time series problem 


- Sale/Price or other timeseries data is used. Like stock prices and you need to predict next _n_ weeks using past weeks data
- Your have columns like `week`,`last_week_price`, `actual`, `predicted_price` etc. Here you will use `last_week_price` and other columns to predict the `predicted_price` in the start of the week.
- After the week is over you will fill the `actual` column and calculate error. Report the error for the week and other stats over last few weeks.


In this scenario if you use file based storage then every week you will read/retrieve entire file. And then overwrite the whole file. But in case you use a SQL storage you will only insert/update few rows for the weeks you are predicting.

**Scenario 3:**

Running Testing of multiple Models over a period of time. Here as well you can either use a file based system where you have a file per model and each day/week/month you keep overwriting the files. Or you can use a sql database and have a table with an extra column `model_used` and just keep adding to table. After sometime a single query can be fired and stats can be collected from the table.


### How to connect and do SQL with Python? 

Just like in java world we a have 2 options
- Use an ORM (Object Relational Mapper) like SQLAlchemy. 
    - This option is good if you intend to build a microservice, store domain entities, and serve customers. Like storing customer info, address, purchases, cart contents etc. Here you want to have classes which Object Oriented programming and these are stored in Databases
- Use direct SQL Connectors and write some lightweight wrappers on top for convinience.
    - This option is good for data-science use cases where our data is mostly denormalized and most use cases involve ad-hoc queries. We use a Database to have incremental updates to our data and auditing.

In this tutorial we will learn to use MySQL with Python and create a simple wrapper that we can use. We will use the MySQL official python package.



### Installing the package and Table Setup

`pip install mysql-connector-python`

I assume you have access to a MySQL installed Machine. In case you don't try with AWS RDS/ AWS Aurora. You can also install mysql on your personal system and try.

#### Connecting to the database

In [1]:
from mysql.connector import MySQLConnection, Error
from datetime import date, datetime, timedelta
import numpy as np # linear algebra
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

conn_details = {"host":"localhost","database":"","user":"root","password":""}

conn = MySQLConnection(**conn_details)
if not conn.is_connected():
    raise AssertionError("Could not connect to DB")
    
    


#### Creating a Database

In [ ]:
create_db_command = "CREATE DATABASE scholar;"

try:
    cursor = conn.cursor()
    cursor.execute(create_db_command)
finally:
    cursor.close()


Notice How we enclosed the query running in a try-finally block. This ensures that we close the cursor even if an exception is thrown. In reality the `conn` variable (opening a connection should be in try-finally as well).

#### Using the database we just created and creating a table for this tutorial

We will create a table for storing students marks in exams. Columns will be 
- student_id(int), 
- date_given(Date), 
- exam_id(int), 
- total(int), 
- score(int), 
- passed(Boolean)

In [4]:
use_db_command = "use scholar;"

create_table_command = """
CREATE TABLE `exam_marks` (
  `student` bigint(11) unsigned NOT NULL,
  `date_given` date NOT NULL,
  `exam_id` int(11) NOT NULL,
  `total` int(11) NOT NULL,
  `score` int(11) DEFAULT NULL,
  `passed` CHAR(1) DEFAULT NULL,
  PRIMARY KEY (`student`,`exam_id`,`date_given`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

"""

try:
    cursor = conn.cursor()
    cursor.execute(use_db_command)
    cursor.execute(create_table_command)
finally:
    cursor.close()

True

### Inserting and Updating Data

### Querying Data

### Our Utility Class

In [32]:
ctr_switch = 0
ctr_no_switch = 0
total = 10000
for i in range(total):
    rn = np.random.randint(1,4)
    guess = np.random.randint(1,4)
    removable = next(iter(set([1,2,3]) - set([guess,rn])))
    remaining = set([1,2,3])-set([removable])
    # we switch
    g2 = next(iter(remaining - set([guess])))
    if g2==rn:
        ctr_switch=ctr_switch+1
    if guess==rn:
        ctr_no_switch=ctr_no_switch+1
        
print("Switch winning probability %s"%(ctr_switch/total))
print("No Switch winning probability %s"%(ctr_no_switch/total))
    

Switch winning probability 0.6716
No Switch winning probability 0.3284
